---
layout: post
title: Sprint 5 Blog
description:  Here's where I will present "Sprint 5 Blog"
courses: {csp: {week: 12}}
type: issues 
comments: true
---

# Stats API Technical Analysis

## 1. List Requests, Use of Lists, Dictionaries, and Database

### List Requests
The Stats API supports the following requests:
- **GET** `/api/stats`: Retrieve the stat entries.
- **POST** `/api/stats`: Add a new stat entry.
- **PUT** `/api/stats`: Update an existing stat entry.
- **DELETE** `/api/stats/<profile_name>/<Guess>`: Delete a specific stat entry.

### Use of Lists and Dictionaries
In the API, we use lists to handle multiple stats entries and dictionaries to represent individual entries. The database rows are converted to dictionaries for JSON responses.

#### Example:


In [ ]:
# Assuming you have a StatsEntry model and a 'stat_value' column
entries = StatsEntry.query.order_by(StatsEntry.stat_value.desc()).all()
entries_list = [entry.read() for entry in entries]


## 2. Formatting Response Data (JSON) from API into DOM
We use Flask's jsonify function to format the response data as JSON. In the frontend, we use JavaScript to fetch the JSON data from the API and update the DOM. This involves converting the JSON response into HTML elements to display the statistics.

#### Example:


In [ ]:
@stats_api.route('/api/stats', methods=['GET'])
def get_stats():
    entries = StatsEntry.query.order_by(StatsEntry.score.desc()).all()
    return jsonify([entry.read() for entry in entries]), 200


## 3. Database Queries
We use SQLAlchemy ORM to interact with the database. SQLAlchemy provides methods to query the database and return results as Python lists. For example, we can retrieve all statistics entries sorted by score or filter entries by profile name.

#### Example:


In [ ]:
# Get all entries sorted by score
entries = StatsEntry.query.order_by(StatsEntry.score.desc()).all()

# Filter by profile name
entry = StatsEntry.query.filter_by(profile_name=profile_name).first()


## 4. CRUD Methods in Class
We define methods in the **StatsEntry** class to perform CRUD operations on the database:

- Create: Adds a new entry to the database.
- Read: Converts a database entry to a dictionary.
- Update: Updates entry fields with new data.
- Delete: Removes an entry from the database.

#### Example:


In [ ]:
class StatsEntry(db.Model):
    # Create
    def create(self):
        db.session.add(self)
        db.session.commit()
        
    # Read
    def read(self):
        return {
            "profile_name": self.profile_name,
            "game_name": self.game_name,
            "score": self.score
        }
        
    # Update
    def update(self, data):
        for key, value in data.items():
            setattr(self, key, value)
        db.session.commit()
        
    # Delete
    def delete(self):
        db.session.delete(self)
        db.session.commit()


## 5. Algorithmic Code Request
We define API endpoints to handle different types of requests. For example, the **PUT** request to update a statistics entry involves checking if the entry exists, updating the score if it does, or creating a new entry if it doesn't.

#### Example:


In [ ]:
@stats_api.route('/api/stats', methods=['PUT'])
def update_stats_entry():
    try:
        data = request.get_json()
        existing_entry = StatsEntry.query.filter_by(
            profile_name=data.get('profile_name'),
            game_name=data.get('game_name')
        ).first()
        
        if existing_entry:
            existing_entry.score = int(data.get('score', 0))
            db.session.commit()
            return jsonify({"message": "Updated"}), 200
        
        return jsonify({"error": "Entry not found"}), 404
    except Exception as e:
        return jsonify({"error": str(e)}), 500


## 6. API Class
We use Flask's **Blueprint** to define the API routes. The **stats_api** blueprint handles the **GET, POST, PUT, and DELETE** methods, allowing us to organize the API endpoints and their implementations.

#### Example:


In [ ]:
stats_api = Blueprint('stats_api', __name__)

@stats_api.route('/api/stats', methods=['GET'])
def get_stats():
    # Implementation

@stats_api.route('/api/stats', methods=['POST'])
def add_stats_entry():
    # Implementation

@stats_api.route('/api/stats', methods=['PUT'])
def update_stats_entry():
    # Implementation

@stats_api.route('/api/stats/<profile_name>/<game_name>', methods=['DELETE'])
def delete_stats_entry(profile_name, game_name):
    # Implementation


## 7. Method with Sequencing, Selection, and Iteration
The **update_stats_entry** method contains sequencing (steps to process the request), selection (conditional checks), and iteration (looping through data if needed). This method updates an existing entry or creates a new one based on the provided data.

#### Example:


In [ ]:
@stats_api.route('/api/stats', methods=['PUT'])
def update_stats_entry():
    try:
        data = request.get_json()
        if not data:
            return jsonify({"error": "No data provided"}), 400

        existing_entry = StatsEntry.query.filter_by(
            profile_name=data.get('profile_name'),
            game_name=data.get('game_name')
        ).first()

        new_score = int(data.get('score', 0))

        if existing_entry:
            existing_entry.score = new_score
            db.session.commit()
            return jsonify({"message": "Score updated successfully"}), 200

        entry = StatsEntry(
            profile_name=data.get('profile_name'),
            game_name=data.get('game_name'),
            score=new_score
        )
        
        if entry.create():
            return jsonify({"message": "New entry created"}), 201
        return jsonify({"error": "Failed to create entry"}), 400

    except Exception as e:
        db.session.rollback()
        return jsonify({"error": str(e)}), 500


## 8. Parameters and Return Type
The **update_stats_entry** method takes JSON data as input and returns a JSON response. The input data includes **profile_name, game_name, and score**. The response is formatted using **jsonify** to ensure it is returned as a JSON object.

#### Example:


In [ ]:
@stats_api.route('/api/stats', methods=['PUT'])
def update_stats_entry():
    try:
        data = request.get_json()
        if not data:
            return jsonify({"error": "No data provided"}), 400

        existing_entry = StatsEntry.query.filter_by(
            profile_name=data.get('profile_name'),
            game_name=data.get('game_name')
        ).first()

        new_score = int(data.get('score', 0))

        if existing_entry:
            existing_entry.score = new_score
            db.session.commit()
            return jsonify({"message": "Score updated successfully"}), 200

        entry = StatsEntry(
            profile_name=data.get('profile_name'),
            game_name=data.get('game_name'),
            score=new_score
        )
        
        if entry.create():
            return jsonify({"message": "New entry created"}), 201
        return jsonify({"error": "Failed to create entry"}), 400

    except Exception as e:
        db.session.rollback()
        return jsonify({"error": str(e)}), 500


## 9. Call to Algorithm Request
In the frontend, we use the **fetch** API to make requests to the backend. For example, to submit a score, we send a **PUT** request with the profile name, game name, and score. The response is handled by checking the status and updating the DOM accordingly. If the request is successful, we update the statistics; if there's an error, we display an error message.

#### Example:


In [ ]:
async function submitScore() {
    const profileName = document.getElementById('profileName').value.trim();
    const gameName = document.getElementById('gameName').value.trim();
    const score = parseInt(document.getElementById('score').value);

    if (!profileName || !gameName) {
        showMessage('Please fill in all fields', true);
        return;
    }

    if (isNaN(score) || score < 0 || score > 100) {
        showMessage('Please enter a valid score ', true);
        return;
    }

    try {
        const response = await fetch(API_URL, {
            method: 'PUT',
            headers: {
                'Content-Type': 'application/json',
            },
            body: JSON.stringify({
                profile_name: profileName,
                game_name: gameName,
                score: score
            })
        });

        const data = await response.json();

        if (response.ok) {
            showMessage(data.message);
            document.getElementById('profileName').value = '';
            document.getElementById('gameName').value = '';
            document.getElementById('score').value = '';
            await fetchStats();
        } else {
            throw new Error(data.error || 'Failed to submit score');
        }
    } catch (error) {
        console.error('Error:', error);
        showMessage(error.message, true);
    }
}
